In [2]:
import cv2
import pandas as pd
import os
import numpy as np
import traceback

# Define the file paths
labels_file_path = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\depth_fixed_occlusion_02.txt' # fixed depth with occlusion
#labels_file_path = r'C:\Users\User\Documents\GitHub\pfas_finalproject\kalman_3d\artificial_occlusion_depth_02.txt' # depth with just occlusion
images_folder_path = r'C:\Users\User\Documents\dataset\pfas\34759_final_project_rect\seq_02\image_02\data'

# Read the labels file with the updated structure
headers = [
    "frame", "track id", "type",
    "bbox_left", "bbox_top", "bbox_right", "bbox_bottom",
    "height_3d", "width_3d", "length_3d",
    "location_x", "location_y", "location_z",
    "rotation_y"
]
df = pd.read_csv(labels_file_path, sep=' ', names=headers)

# Define constants
dt = 0.1036  # Time difference between frames (adjust based on your data)
max_distance = 100  # Adjust as needed
max_missing_frames = 40  # Adjust as needed

# Initialize variables
missing_ids = []
tracked_ids = []
new_ids = []
locations_dict = {}       # Stores bounding boxes for each track_id
sizes_dict = {}           # Stores width and height for each track_id
depths_dict = {}          # Stores depth for each track_id
object_types_dict = {}    # Stores object types for each track_id
previous_frame_ids = []
previous_frame_locations = {}
frame_counter = 1
reassociation_map = {}
kalman_filters = {}
missing_counts = {}

# Get sorted list of frames
unique_frames = sorted(df["frame"].unique())

# Kalman Filter Class Definition with Acceleration in All Dimensions
class KalmanFilter3D:
    def __init__(self, initial_state, dt):
        self.state_dim = 9  # x, y, z, vx, vy, vz, ax, ay, az
        self.meas_dim = 3   # x, y, z (depth)
        self.dt = dt

        # Initial state vector
        self.x = np.zeros((self.state_dim, 1))
        self.x[0:3, 0] = initial_state[0:3]  # Initialize position (x, y, z)

        # State Transition Matrix
        dt = self.dt
        self.F = np.array([
            [1, 0, 0, dt, 0,  0,  0.5 * dt**2, 0,           0],
            [0, 1, 0, 0,  dt, 0,  0,           0.5 * dt**2, 0],
            [0, 0, 1, 0,  0,  dt, 0,           0,           0.5 * dt**2],
            [0, 0, 0, 1,  0,  0,  dt,          0,           0],
            [0, 0, 0, 0,  1,  0,  0,           dt,          0],
            [0, 0, 0, 0,  0,  1,  0,           0,           dt],
            [0, 0, 0, 0,  0,  0,  1,           0,           0],
            [0, 0, 0, 0,  0,  0,  0,           1,           0],
            [0, 0, 0, 0,  0,  0,  0,           0,           1],
        ])

        # Observation Matrix
        self.H = np.zeros((self.meas_dim, self.state_dim))
        self.H[0, 0] = 1  # x position
        self.H[1, 1] = 1  # y position
        self.H[2, 2] = 1  # z position (depth)

        # Process Noise Covariance
        q = 25.0  # Process noise magnitude (tune as needed)
        dt4 = (dt ** 4) / 4
        dt3 = (dt ** 3) / 2
        dt2 = dt ** 2
        q_block = np.array([
            [dt4,    0,    0, dt3,    0,    0, dt2/2,    0,    0],
            [0,    dt4,    0,    0, dt3,    0,    0, dt2/2,    0],
            [0,       0, dt4,    0,    0, dt3,    0,    0, dt2/2],
            [dt3,    0,    0, dt2,    0,    0, dt,       0,    0],
            [0,    dt3,    0,    0, dt2,    0,    0,     dt,    0],
            [0,       0, dt3,    0,    0, dt2,    0,    0,    dt],
            [dt2/2,  0,    0, dt,     0,    0, 1,        0,    0],
            [0,  dt2/2,    0,    0, dt,     0,    0,    1,     0],
            [0,       0, dt2/2, 0,    0,  dt,    0,     0,     1],
        ])
        self.Q = q * q_block

        # Measurement Noise Covariance
        r = 10.0  # Measurement noise variance for position (tune as needed)
        self.R = np.eye(self.meas_dim) * r

        # Initial covariance matrix
        self.P = np.eye(self.state_dim) * 1000

        # Flag to indicate if the filter was updated with a measurement
        self.updated_with_measurement = False

    def predict(self):
        self.x = np.dot(self.F, self.x)
        self.P = np.dot(self.F, np.dot(self.P, self.F.T)) + self.Q
        self.updated_with_measurement = False

    def update(self, measurement):
        z = np.array(measurement).reshape(-1, 1)  # measurement includes x, y, z
        # Measurement residual
        y = z - np.dot(self.H, self.x)
        # Residual covariance
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R
        # Kalman gain
        K = np.dot(self.P, np.dot(self.H.T, np.linalg.inv(S)))
        # Update state estimate
        self.x = self.x + np.dot(K, y)
        # Update covariance estimate
        self.P = self.P - np.dot(K, np.dot(self.H, self.P))
        self.updated_with_measurement = True

    def get_current_state(self):
        return self.x

    def get_predicted_location(self):
        return self.x[:3, 0]  # Return x, y, z positions

def get_frame_data(frame_id):
    frame_data = df[df["frame"] == frame_id]
    frame_ids = frame_data["track id"].unique().tolist()
    frame_locations = {}
    frame_depths = {}
    frame_object_types = {}
    for _, row in frame_data.iterrows():
        track_id = row["track id"]
        bbox = [row["bbox_left"], row["bbox_top"], row["bbox_right"], row["bbox_bottom"]]
        depth = row["location_z"]  # Assuming depth is in location_z
        object_type = row["type"]
        frame_locations[track_id] = bbox
        frame_depths[track_id] = depth
        frame_object_types[track_id] = object_type
    return frame_ids, frame_locations, frame_depths, frame_object_types

def track_states(current_frame_ids, previous_frame_ids):
    missing_ids = [id_ for id_ in previous_frame_ids if id_ not in current_frame_ids]
    tracked_ids = [id_ for id_ in current_frame_ids if id_ in previous_frame_ids]
    new_ids = [id_ for id_ in current_frame_ids if id_ not in previous_frame_ids]
    return missing_ids, tracked_ids, new_ids

def compute_distance_3d(pos1, pos2):
    # Compute Euclidean distance between two 3D points
    return np.linalg.norm(np.array(pos1) - np.array(pos2))

def match_new_ids_to_missing_predictions(new_ids, missing_ids_prediction, current_frame_locations, current_frame_depths, max_distance):
    matched_ids = []
    reassociation_map = {}
    unmatched_new_ids = []

    for new_id in new_ids:
        min_distance = float('inf')
        best_match_id = None
        new_bbox = current_frame_locations[new_id]
        new_depth = current_frame_depths[new_id]
        new_center = calculate_center(new_bbox)
        new_pos = [new_center[0], new_center[1], new_depth]

        for missing_id, predicted_state in missing_ids_prediction.items():
            predicted_pos = predicted_state[:3, 0]
            distance = compute_distance_3d(new_pos, predicted_pos)
            if distance < min_distance:
                min_distance = distance
                best_match_id = missing_id

        if min_distance < max_distance:
            matched_ids.append((best_match_id, new_id))
            reassociation_map[new_id] = best_match_id
            # Remove matched missing_id from missing_ids_prediction
            missing_ids_prediction.pop(best_match_id)
        else:
            unmatched_new_ids.append(new_id)

    return matched_ids, reassociation_map, unmatched_new_ids

def extract_bbox_from_state(state, size):
    x_center = state[0, 0]
    y_center = state[1, 0]
    width, height = size
    x_top_left = x_center - width / 2
    y_top_left = y_center - height / 2
    x_bottom_right = x_center + width / 2
    y_bottom_right = y_center + height / 2
    return [x_top_left, y_top_left, x_bottom_right, y_bottom_right]

def calculate_center(bbox):
    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2
    return np.array([x_center, y_center])

# Main processing loop
for frame in unique_frames:
    try:
        print(f"Processing frame {frame}")
        # Clear update tracking lists
        updated_ids = []
        predicted_only_ids = []
        about_to_remove_ids = []

        # Step 2: Retrieve Frame Data
        current_frame_ids, current_frame_locations, current_frame_depths, current_frame_object_types = get_frame_data(frame)
        print(f"Current frame IDs: {current_frame_ids}")

        # Update object_types_dict with current frame's object types
        object_types_dict.update(current_frame_object_types)

        # Read the corresponding image
        image_path = os.path.join(images_folder_path, f"{int(frame):010d}.png")
        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            continue

        image = cv2.imread(image_path)

        # Verify if the image has been loaded correctly
        if image is None or image.size == 0:
            print(f"Error loading image: {image_path}")
            continue

        frame_height, frame_width = image.shape[:2]

        # Step 3: Determine States (track_states)
        missing_ids, tracked_ids, new_ids = track_states(current_frame_ids, previous_frame_ids)
        print(f"Missing IDs: {missing_ids}")
        print(f"Tracked IDs: {tracked_ids}")
        print(f"New IDs: {new_ids}")

        # Update missing_counts for missing_ids
        for id_ in missing_ids:
            missing_counts[id_] = missing_counts.get(id_, 0) + 1

        # Step 4: Process missing_ids
        missing_ids_prediction = {}
        for missing_id in missing_ids.copy():
            if missing_id in kalman_filters:
                # Predict the position
                kalman_filter = kalman_filters[missing_id]
                kalman_filter.predict()
                predicted_state = kalman_filter.get_current_state()
                missing_ids_prediction[missing_id] = predicted_state  # Store predicted state
                predicted_only_ids.append(missing_id)

                # Update locations_dict with predicted bbox
                size = sizes_dict.get(missing_id, (0, 0))
                predicted_bbox = extract_bbox_from_state(predicted_state, size)
                locations_dict[missing_id] = predicted_bbox
                # Update depths_dict with predicted depth
                depths_dict[missing_id] = predicted_state[2, 0]
            else:
                print(f"Kalman filter not found for missing ID {missing_id}")
                missing_ids.remove(missing_id)

        # Step 5: Match new_ids to missing_ids_prediction and Initialize Kalman Filters for Unmatched new_ids
        matched_ids, reassociation_map, unmatched_new_ids = match_new_ids_to_missing_predictions(
            new_ids, missing_ids_prediction.copy(), current_frame_locations, current_frame_depths, max_distance
        )
        print(f"Matched IDs: {matched_ids}")
        print(f"Unmatched New IDs: {unmatched_new_ids}")

        # Initialize Kalman filters for unmatched new_ids
        for new_id in unmatched_new_ids:
            bbox = current_frame_locations[new_id]
            center = calculate_center(bbox)
            depth = current_frame_depths[new_id]
            measurement = [center[0], center[1], depth]
            kalman_filter = KalmanFilter3D(initial_state=measurement, dt=dt)
            kalman_filters[new_id] = kalman_filter
            sizes_dict[new_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])  # Store width and height
            depths_dict[new_id] = depth  # Store depth
            if new_id not in tracked_ids:
                tracked_ids.append(new_id)
            locations_dict[new_id] = bbox
            updated_ids.append(new_id)  # Newly initialized, considered updated

        # Remove matched missing_ids from missing_ids and missing_counts
        for missing_id, new_id in matched_ids:
            if missing_id in missing_ids:
                missing_ids.remove(missing_id)
            if missing_id in missing_counts:
                del missing_counts[missing_id]
            # Update Kalman filter mapping if necessary
            if missing_id in kalman_filters:
                pass
            else:
                kalman_filters[missing_id] = kalman_filters.pop(new_id)
            # Update sizes_dict
            sizes_dict[missing_id] = sizes_dict.get(missing_id, sizes_dict.get(new_id))
            if new_id in sizes_dict:
                del sizes_dict[new_id]
            # Update locations_dict
            locations_dict[missing_id] = current_frame_locations[new_id]
            if new_id in locations_dict:
                del locations_dict[new_id]
            # Update depths_dict
            depths_dict[missing_id] = current_frame_depths[new_id]
            if new_id in depths_dict:
                del depths_dict[new_id]
            # Update object_types_dict
            object_types_dict[missing_id] = object_types_dict.get(new_id, "misc")
            if new_id in object_types_dict:
                del object_types_dict[new_id]
            # Update Kalman filter with new measurement
            kalman_filter = kalman_filters[missing_id]
            kalman_filter.predict()
            bbox = current_frame_locations[new_id]
            center = calculate_center(bbox)
            depth = current_frame_depths[new_id]
            measurement = [center[0], center[1], depth]
            kalman_filter.update(measurement)
            updated_ids.append(missing_id)  # Mark as updated with measurement

        # Step 6: Process tracked_ids
        matched_missing_ids = [m[0] for m in matched_ids]
        for tracked_id in tracked_ids.copy():
            if tracked_id in matched_missing_ids:
                continue  # Already updated in Step 5

            if tracked_id not in kalman_filters:
                bbox = current_frame_locations.get(tracked_id, [0, 0, 0, 0])
                center = calculate_center(bbox)
                depth = current_frame_depths.get(tracked_id, 0)
                measurement = [center[0], center[1], depth]
                kalman_filter = KalmanFilter3D(initial_state=measurement, dt=dt)
                kalman_filters[tracked_id] = kalman_filter
                sizes_dict[tracked_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])
                depths_dict[tracked_id] = depth
                print(f"Initialized Kalman filter for tracked ID {tracked_id}")

            kalman_filter = kalman_filters[tracked_id]
            # Predict step
            kalman_filter.predict()

            if tracked_id in current_frame_locations:
                # Update step with measurement
                bbox = current_frame_locations[tracked_id]
                center = calculate_center(bbox)
                depth = current_frame_depths[tracked_id]
                measurement = [center[0], center[1], depth]
                kalman_filter.update(measurement)
                # Update sizes_dict with the new size
                sizes_dict[tracked_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])
                # Update depths_dict with the new depth
                depths_dict[tracked_id] = depth
                # Update locations_dict with the updated bbox
                locations_dict[tracked_id] = bbox
                updated_ids.append(tracked_id)  # Mark as updated
            else:
                # Handle predictions for tracked IDs not in current frame
                predicted_state = kalman_filter.get_current_state()
                size = sizes_dict.get(tracked_id, (0, 0))
                predicted_bbox = extract_bbox_from_state(predicted_state, size)
                locations_dict[tracked_id] = predicted_bbox
                depths_dict[tracked_id] = predicted_state[2, 0]
                predicted_only_ids.append(tracked_id)

        # Step 7: Update Tracking States for Next Frame
        previous_frame_ids = tracked_ids + missing_ids
        previous_frame_locations = {id_: locations_dict[id_] for id_ in previous_frame_ids if id_ in locations_dict}

        # Remove stale missing_ids
        for missing_id in missing_ids.copy():
            if missing_counts[missing_id] > max_missing_frames:
                # Remove object from tracking
                print(f"Removed Missing ID {missing_id} after exceeding max missing frames")
                missing_ids.remove(missing_id)
                del kalman_filters[missing_id]
                del missing_counts[missing_id]
                locations_dict.pop(missing_id, None)
                sizes_dict.pop(missing_id, None)
                depths_dict.pop(missing_id, None)
                object_types_dict.pop(missing_id, None)

        # Increment frame_counter
        frame_counter += 1

        # Visualization
        # Define a mapping of object types to colors for classification
        classification_colors = {
            "Pedestrian": (34, 139, 34),  # Green
            "Car": (0, 0, 128),         # Navy Blue
            "Cyclist": (139, 0, 0),     # Dark Red
            "misc": (128, 0, 128)       # Purple (for unclassified objects)
        }

        # Draw bounding boxes
        for track_id, bbox in locations_dict.items():
            x_top_left, y_top_left, x_bottom_right, y_bottom_right = bbox

            # Get the object type for the current track_id
            object_type = object_types_dict.get(track_id, "misc")
            color = classification_colors.get(object_type, (128, 128, 128))  # Default to gray if type is unknown

            # Draw the bounding box with the classification color
            cv2.rectangle(image, (int(x_top_left), int(y_top_left)), (int(x_bottom_right), int(y_bottom_right)), color, 2)

            # Get depth and whether it's predicted or measured
            kalman_filter = kalman_filters[track_id]
            if kalman_filter.updated_with_measurement:
                depth = depths_dict.get(track_id, 0)
                depth_text = f"d: {depth:.2f}m"
            else:
                depth = depths_dict.get(track_id, 0)
                depth_text = f"pd: {depth:.2f}m"

            # Draw text label with background rectangle
            text = f"ID: {track_id}, {depth_text}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            thickness = 1
            text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
            text_width, text_height = text_size
            # Draw rectangle behind text
            cv2.rectangle(image, (int(x_top_left), int(y_top_left) - text_height - 5),
                          (int(x_top_left) + text_width, int(y_top_left)), color, -1)
            # Draw text
            cv2.putText(image, text, (int(x_top_left), int(y_top_left) - 5), font, font_scale, (255, 255, 255), thickness)

            # Draw the predicted center point
            predicted_center = kalman_filter.get_current_state()[:2, 0]

            # Draw predicted center (blue circle)
            cv2.circle(image, (int(predicted_center[0]), int(predicted_center[1])), 5, (255, 0, 0), -1)  # Blue for predicted center

            # If updated with measurement, draw measurement point (yellow circle)
            if kalman_filter.updated_with_measurement:
                measurement_center = kalman_filter.x[:2, 0]
                cv2.circle(image, (int(measurement_center[0]), int(measurement_center[1])), 5, (0, 255, 255), -1)  # Yellow for measurement

        # Add frame number to the top-left corner with a background
        frame_text = f"Frame: {frame}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        text_color = (255, 255, 255)  # White text
        bg_color = (0, 0, 0)  # Black background

        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(frame_text, font, font_scale, font_thickness)

        # Set position for text and background
        text_x = 10
        text_y = 30
        bg_x1 = text_x - 5
        bg_y1 = text_y - text_height - 5
        bg_x2 = text_x + text_width + 5
        bg_y2 = text_y + baseline

        # Draw the background rectangle
        cv2.rectangle(image, (bg_x1, bg_y1), (bg_x2, bg_y2), bg_color, -1)

        # Put the text on top of the background
        cv2.putText(image, frame_text, (text_x, text_y), font, font_scale, text_color, font_thickness)

        # Show the image with the bounding boxes
        cv2.imshow("Image", image)
        key = cv2.waitKey(100) & 0xFF
        if key == ord('q'):
            break

        # Update previous_frame_ids for the next iteration
        previous_frame_ids = tracked_ids + missing_ids

    except Exception as e:
        print(f"Exception occurred at frame {frame}: {e}")
        traceback.print_exc()
        break

cv2.destroyAllWindows()


Processing frame 0
Current frame IDs: [0, 1, 2, 4, 7, 23]
Missing IDs: []
Tracked IDs: []
New IDs: [0, 1, 2, 4, 7, 23]
Matched IDs: []
Unmatched New IDs: [0, 1, 2, 4, 7, 23]
Processing frame 1
Current frame IDs: [0, 1, 2, 4, 7, 23]
Missing IDs: []
Tracked IDs: [0, 1, 2, 4, 7, 23]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 2
Current frame IDs: [0, 1, 2, 4, 7, 23]
Missing IDs: []
Tracked IDs: [0, 1, 2, 4, 7, 23]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 3
Current frame IDs: [0, 1, 2, 7, 23]
Missing IDs: [4]
Tracked IDs: [0, 1, 2, 7, 23]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 4
Current frame IDs: [0, 1, 2, 7, 23]
Missing IDs: [4]
Tracked IDs: [0, 1, 2, 7, 23]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 5
Current frame IDs: [0, 1, 2, 7, 23]
Missing IDs: [4]
Tracked IDs: [0, 1, 2, 7, 23]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 6
Current frame IDs: [0, 1, 2, 7, 23]
M